In [1]:
import pandas as pd
import numpy as np

import random
from itertools import combinations, permutations
from functools import reduce
from collections import defaultdict

This one was challenging requiring quite a bit of forethought. In particular, the challenge for me was thinking of an efficient way to generate the subsets of numbers. I used a sort of backtracking approach to do it quickly and create unique subsets of the numbers.

My code does the following:
1. ***Primes.*** Generate all primes that have unique digits (i.e., each digit only appears once). It does not need to contain every digit $1$ to $9$, but it can only contain them once. This is the slowest part of the code.
2. ***Organize Lookup Tables.*** For each prime, save the tuple associated with them. So for example, $4231$ is prime and the associated tuple is $(1,2,3,4)$; the same is true for $2341$. Additionally, for each tuple, associate it with the minimum value of the tuple. So for $(1,2,3,4)$, the minimum value is $1$. This allows us to quickly count up all primes for a given tuple, and the minimum key allows us to do it uniquely (i.e., in increasing order).
3. ***Recursion.*** Start with the set $\{1,2,3,4,5,6,7,8,9\}$. Then take the minimum value of the set--in the first iteration, it is $1$. Then go through each tuple under the $1$ minimum key, store it, and remove it from the original set and continue recursively--for the first iteration, if the first tuple is $(1,4)$ then the recursion would be called on the remaining numbrers $\{2,3,5,6,7,8,9\}$, and the next minimum value would be $2$. If you reach a point where there are no set elements remaining, that means you have completed and found a group of tuples that form a pandigital prime set! If there are no tuples that work, then you send back an empty signaling that this path failed.
4. ***Answer.*** To generate the answer, you have to get all valid pandigital prime sets. Then for each of those prime sets, you go through each tuple within them, and multiply by the number of primes associated with each tuple. For example, if the set is $[(1,3), (2)]$, then you would multiply $2 \times 1$: $2$ because $(1,3)$ could be $13$ or $31$ which are both prime, and $1$ because $(2)$ can only be $2$.

The prime generation is the slowest part of the code--even with some small optimizations (not allowing even only sets, removing divisible by 3, etc.). If you can speed up prime generation of primes with unique digits, then the code can do better. However, the whole thing runs in ~3-4 secs which I am happy with.

In [2]:
# helper functions to check primes quickly
def sieve(n):
    arr = [0,0,1] + [1,0]*(n//2 + 1)
    i = 3
    while i*i <= n:
        if arr[i]:  
            arr[i*i::2*i] = [0]*len(arr[i*i::2*i])
        i += 2

    ret = []
    for (i, p) in enumerate(arr):
        if p:
            ret.append(i)

    return arr, ret

def miller_rabin(n, k = 10):
    if n <= 3:
        return n == 2 or n == 3

    if n % 2 == 0:
        return False

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

pbs, ps = sieve(10**6)
def prime_check(n):
    if n < 10**6:
        return pbs[n] == 1

    return miller_rabin(n)
    

In [3]:
# find every prime number combination, by tuple, with only one of each digit (no repeats)
# does this up to 8 digits, since there are no pandigital prime 9 digit numbers 
# (digits add to multiple of 3 by definition)
primes = defaultdict(lambda: [])
primes[(2,)].append(2)
primes[(3,)].append(3)
primes[(5,)].append(5)
primes[(7,)].append(7)

for digs in range(2,9):
    for nums in combinations(range(1,10), digs):
        # skip combinations of only even numbers
        if len(set(nums) - {2,4,6,8}) == 0:
            continue
        
        # skip if divisible by 3
        if sum(nums) % 3 == 0:
            continue

        for order in permutations(range(digs)):
            number = reduce(lambda a,b: a + nums[b]*(10**(order[b])), range(digs), 0)

            if prime_check(number):
                primes[nums].append(number)

# for each prime tuple, use it's minimum number as key, to avoid duplicates
tups = {i: defaultdict(lambda: []) for i in range(1,10)}
for key in primes:
    tups[min(key)][len(key)].append(key)

# recursively form all lists of tuples of numbers that would form pandigital prime
def prime_sets(target = {1,2,3,4,5,6,7,8,9}, sets_so_far = []):    
    set_seen = reduce(lambda a,b: a.union(b), sets_so_far, set())

    if len(target) - len(set_seen) < 0:
        return []

    if len(target) - len(set_seen) == 0:
        return sets_so_far
    
    # if there's a single prime left, just do a manual check
    if len(target) - len(set_seen) == 1:
        if 2 in target and 2 not in set_seen:
            return sets_so_far + [(2,)]
        if 3 in target and 3 not in set_seen:
            return sets_so_far + [(3,)]
        if 5 in target and 5 not in set_seen:
            return sets_so_far + [(5,)]
        if 7 in target and 7 not in set_seen:
            return sets_so_far + [(7,)]
        
        return []

    tups_to_check = min(target - set_seen)
    ts = tups[tups_to_check]

    rets = []
    # go in increasing order
    for i in ts.keys():
        for tup in ts[i]:
            # if you see a number you've already seen
            if len(set_seen.intersection(tup)) > 0:
                continue

            # if tup contains a number that isn't supposed to be there
            if target.union(tup) != target:
                continue

            # get all the the sub results of the smaller problem
            possibles = prime_sets(target, sets_so_far + [tup])

            # if it's a single list, wrap it in another list, to keep consistency
            if len(possibles) > 0 and type(possibles[0]) == tuple:
                possibles = [possibles]
            
            # for each possible set of tuples
            for possible in possibles:
                # reduce it to a set
                set_possible = reduce(lambda a,b: a.union(b), possible, set())
                # confirm that it works as a tuple
                if set_possible == target:
                    # append it to final list
                    rets.append(sorted(possible))

    return rets

prime_sets({1,2,3,4})

[[(1, 4), (2,), (3,)],
 [(1, 4), (2, 3)],
 [(1, 2, 4), (3,)],
 [(1, 3, 4), (2,)],
 [(1, 2, 3, 4)]]

In [4]:
pss = prime_sets()
s = 0
for ps in pss:
    to_add = 1
    for sub in ps:
        to_add *= len(primes[sub])
    
    s += to_add

print(s)

44680
